[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NU-MSE-LECTURES/465_Computational_Microscopy_2026/blob/dev/Week_03/exercises/exercise_03_image_analysis_basics.ipynb)

# Week 3 Exercises: Image Analysis Basics
**Objective:** Reinforce fundamental image processing concepts with practical exercises.

## Instructions
1. Attempt the exercises in the cells below.
2. Check your understanding by running the code and visualizing results.
3. Modify parameters and observe how they affect the output.
4. If you get stuck, review the corresponding code examples in the `code_examples/` folder.

In [ ]:
# Colab setup
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab. Installing requirements...")
    !pip install hyperspy scikit-image scipy pandas matplotlib
    !git clone https://github.com/NU-MSE-LECTURES/465_Computational_Microscopy_2026.git
    print("Setup complete.")
except ImportError:
    IN_COLAB = False
    print("Not running in Google Colab.")

# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import filters, io, data, exposure, morphology, measure, segmentation

# Set random seed for reproducibility
np.random.seed(42)

## Exercise 1: Filtering Comparison

You are given a noisy image with Gaussian and salt-and-pepper noise.

**Task:**
1. Load the `astronaut` image from scikit-image.
2. Convert to grayscale and add Gaussian noise (std=20) and salt-and-pepper noise (10% contamination).
3. Apply three filters: Gaussian, Median, and bilateral filters.
4. Calculate the MSE (mean squared error) between the filtered image and the original (noisy) image.
5. Visualize the results in a 2x3 subplot grid (original + noise, then three filtered versions).
6. **Which filter performs best for this type of noise?**

In [ ]:
# Your code here
# Hint: Use skimage.data.astronaut(), skimage.color.rgb2gray()
# Hint: Add noise using: image + np.random.normal(0, 20, image.shape)
# Hint: For salt-and-pepper, randomly set pixels to 0 or 255
# Hint: Use filters.gaussian(), filters.median(), filters.bilateral()
# Hint: MSE = np.mean((img1 - img2)**2)


## Exercise 2: FFT Analysis

Analyze the frequency content of an image with periodic noise.

**Task:**
1. Create a simple synthetic image (e.g., a checkerboard or sine wave pattern).
2. Add periodic (sinusoidal) noise by superimposing a sine wave at a specific frequency.
3. Compute the 2D FFT of the noisy image.
4. Visualize the magnitude spectrum (in log scale) and identify the frequency peaks corresponding to the noise.
5. Apply a notch filter centered at the noise frequency and visualize the filtered result.
6. **Compare the original vs filtered image in both spatial and frequency domains.**

In [ ]:
# Your code here
# Hint: Create synthetic image: X, Y = np.meshgrid(...); image = np.sin(2*np.pi*X/period)
# Hint: Add periodic noise: image += 0.2 * np.sin(2*np.pi*X/10)
# Hint: Compute FFT: F = np.fft.fft2(image); Fshift = np.fft.fftshift(F)
# Hint: Magnitude spectrum: mag = np.abs(Fshift); log_mag = np.log(1 + mag)
# Hint: Create notch filter as a circular mask around the peak frequency
# Hint: Apply filter: F_filtered = Fshift * notch_mask; inverse FFT to get filtered image


## Exercise 3: Histogram Equalization

Enhance contrast in a low-contrast image using different methods.

**Task:**
1. Create a low-contrast synthetic image (e.g., a gradient or mixture of gaussians with narrow intensity range).
2. Apply three enhancement techniques:
   - Global histogram equalization
   - CLAHE (Adaptive Histogram Equalization) with different clip_limit values (0.01, 0.03, 0.1)
3. For each result, plot the original and enhanced histograms side-by-side.
4. Visualize the original and all enhanced images in a grid.
5. **Which method provides the best contrast without oversaturation? Why?**

In [ ]:
# Your code here
# Hint: Create low-contrast image: image = np.random.normal(128, 20, (100, 100))
# Hint: Clip to valid range: image = np.clip(image, 0, 255).astype(np.uint8)
# Hint: Global equalization: exposure.equalize_hist(image)
# Hint: CLAHE: exposure.equalize_adapthist(image, kernel_size=tile_size, clip_limit=clip_limit)
# Hint: Plot histogram: plt.hist(image.ravel(), bins=256, range=(0, 256))


## Exercise 4: Thresholding Methods

Compare different thresholding strategies on a microscopy-like image.

**Task:**
1. Create a synthetic "particle" image (multiple circular/elliptical objects on a background with varying intensity).
2. Implement and apply four thresholding methods:
   - Manual threshold (fixed value)
   - Otsu's method
   - Local adaptive thresholding (block_size=31)
   - Niblack thresholding
3. Count the number of connected components (particles) detected by each method.
4. Visualize original image and all thresholded results in a grid.
5. **Which method is most robust to intensity variations?**

In [ ]:
# Your code here
# Hint: Create particles: use scipy.ndimage.gaussian_filter on random point coordinates
# Hint: Manual threshold: binary = image > 128
# Hint: Otsu: threshold = filters.threshold_otsu(image); binary = image > threshold
# Hint: Local: binary = image > filters.threshold_local(image, block_size=31)
# Hint: Niblack: binary = image > filters.threshold_niblack(image, window_size=31)
# Hint: Count components: ndimage.label(binary)[1] gives the count


## Exercise 5: Morphological Operations

Clean up a segmented image using morphological operations.

**Task:**
1. Start with a noisy binary image (checkerboard pattern with random noise).
2. Apply the following operations in sequence and visualize each step:
   - Opening (erosion followed by dilation)
   - Closing (dilation followed by erosion)
   - Remove small objects (min_size=50)
   - Fill small holes (hole_size=50)
3. At each step, count the number of connected components and the total area (number of white pixels).
4. Plot the progression in a grid showing the effect of each operation.
5. **How do opening and closing affect the overall structure differently?**

In [ ]:
# Your code here
# Hint: Create noisy binary: binary = np.random.rand(200, 200) > 0.5; then threshold
# Hint: Opening: morphology.opening(binary, selem=morphology.disk(5))
# Hint: Closing: morphology.closing(binary, selem=morphology.disk(5))
# Hint: Remove small objects: morphology.remove_small_objects(binary, min_size=50)
# Hint: Fill holes: ndimage.binary_fill_holes(binary)
# Hint: Count components: label_image, num_components = ndimage.label(binary)
# Hint: Total area: np.sum(binary)


## Exercise 6: Watershed Segmentation

Separate touching particles using watershed segmentation.

**Task:**
1. Create a synthetic image with overlapping circular particles (simulating clustering).
2. Threshold the image to get a binary mask.
3. Apply watershed segmentation:
   - Compute the distance transform of the binary mask.
   - Find local maxima to identify particle centers.
   - Use these as markers for the watershed algorithm.
4. Color-label the segmented regions and overlay on the original image.
5. Compare the number of particles detected before (simple labeling) and after watershed.
6. **Does watershed correctly separate overlapping particles?**

In [ ]:
# Your code here
# Hint: Create overlapping particles by drawing circles with ndimage.gaussian_filter
# Hint: Threshold: binary = image > filters.threshold_otsu(image)
# Hint: Distance transform: dist = ndimage.distance_transform_edt(binary)
# Hint: Find peaks: coords = feature.peak_local_max(dist, min_distance=10, labels=binary)
# Hint: Create markers: markers = ndimage.label(coords)[0]
# Hint: Watershed: labeled = segmentation.watershed(dist_inv, markers=markers, mask=binary)
# Hint: Use plt.imshow with cmap='nipy_spectral' to color different regions


## Exercise 7: Image Properties & Quantification

Extract morphological properties from segmented particles.

**Task:**
1. Segment an image (from previous exercise or new) to get labeled regions.
2. Use `skimage.measure.regionprops()` to extract properties for each particle:
   - Area
   - Eccentricity (circularity)
   - Solidity (how "full" the bounding box is)
   - Equivalent diameter
3. Create a summary table (pandas DataFrame) with these properties.
4. Plot distributions of each property (histogram or violin plot).
5. Identify and visualize particles that are significantly different from the population mean.
6. **Can you classify particles as "round" vs "elongated" based on eccentricity?**

In [ ]:
# Your code here
# Hint: Get properties: props = measure.regionprops(labeled_image)
# Hint: Extract individual properties: areas = [prop.area for prop in props]
# Hint: Create DataFrame: import pandas as pd; df = pd.DataFrame({...})
# Hint: Filter round particles: round_particles = df[df['eccentricity'] < 0.5]
# Hint: Visualize on original image: highlight regions with imshow using cmap


## Challenge: Build a Complete Pipeline

Combine techniques from Exercises 1-7 into a single image analysis pipeline.

**Task:**
1. Load a real microscopy image (use the GaAs_IDB_1.emd from Week 3 assignment, or create a synthetic one).
2. Build a pipeline:
   - **Step 1 (Preprocessing):** Apply noise reduction (Gaussian or Median filter).
   - **Step 2 (Enhancement):** Improve contrast using CLAHE or histogram equalization.
   - **Step 3 (Segmentation):** Threshold (Otsu or adaptive) and apply morphological operations.
   - **Step 4 (Separation):** Use watershed for overlapping regions.
   - **Step 5 (Quantification):** Extract morphological metrics for each particle.
3. Create a comprehensive visualization showing all steps.
4. Export results to a CSV file with particle properties.
5. **Justify each step: Why did you choose that particular filter, threshold method, or enhancement?**

In [ ]:
# Your code here
# Build a complete pipeline as a function:
# def analyze_image(image_path, filter_sigma=1.0, clahe_clip=0.03, threshold_method='otsu'):
#     # Load image
#     # Apply filters
#     # Enhance contrast
#     # Segment
#     # Quantify
#     # Return results
